<a href="https://colab.research.google.com/github/giangkarry/Machine-Learning/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/giangkarry/Machine-Learning.git

In [2]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report

In [3]:
classes = ('2C', '3C', '4C')
TrainTest = namedtuple('TrainTest', ['train', 'test'])
traindir = '/content/Machine-Learning/DATA_CHAMBER_2021/train'
testdir = '/content/Machine-Learning/DATA_CHAMBER_2021/test'
#hàm chuẩn bị dữ liệu
def prepare_data():
  input_size = 224
  transform = transforms.Compose([
        transforms.Resize((input_size,input_size)), 
        transforms.ToTensor()
    ])
  trainset = torchvision.datasets.ImageFolder(root= traindir, transform=transform)
  testset  = torchvision.datasets.ImageFolder(root=testdir, transform=transform)

  return TrainTest(
      train=trainset,
      test=testset
  )

#hàm chuẩn bị dữ liệu theo batch đưa vào model
def prepare_loader(datasets):
    batch_size = 32
    num_workers = 4
    trainloader = torch.utils.data.DataLoader(
        dataset=datasets.train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(
        dataset=datasets.test, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return TrainTest(
        train=trainloader,
        test=testloader
    )


In [4]:
#train
def get_trainer(model):
  loss = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  return loss, optimizer

#hàm train trong mỗi epoch
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
    model.train()
    running_loss = 0.0
    reporting_step = 42
    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % reporting_step == reporting_step-1:
            print(f"Epoch {epoch} Step {i} ave_loss {running_loss/reporting_step:0.4f}")
            running_loss = 0.0

In [5]:
#hàm test
def test_epoch(epoch, model, loader, device):
    model.eval()
    ypred = []
    ytrue = []
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ypred += list(predicted.cpu().numpy())
      ytrue += list(labels.cpu().numpy())
    return ypred, ytrue

In [6]:
#thực thi
def main(model = 'vgg16'):
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  print("Tập train: ", len(datasets.train))
  print("Tập test: ", len(datasets.test))
  print("class: ", datasets.test.class_to_idx)
  
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  if model == 'vgg16':
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  
  elif (model == 'vgg19'):
    model = torchvision.models.vgg19()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  
  elif model == 'resnet50':
    model = torchvision.models.resnet50()
    model.fc = torch.nn.modules.Linear(in_features=2048, out_features=3)

  n_epoch = 0
  model.to(device)
  print(device)

  loss, optimizer = get_trainer(model)
  for epoch in range(n_epoch):
        train_epoch(epoch, model, loaders.train, loss, optimizer, device)
        ypred, ytrue = test_epoch(epoch, model, loaders.test, device)
        print(classification_report(ytrue, ypred, target_names=classes))

In [7]:
#VGG16
main('vgg16')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Tập train:  6717
Tập test:  1607
class:  {'2C': 0, '3C': 1, '4C': 2}
cuda:0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0 Step 41 ave_loss 1.0755
Epoch 0 Step 83 ave_loss 1.0730
Epoch 0 Step 125 ave_loss 0.6786
Epoch 0 Step 167 ave_loss 0.3786
Epoch 0 Step 209 ave_loss 0.2959
              precision    recall  f1-score   support

          2C       0.80      0.68      0.73       409
          3C       0.55      0.99      0.70       367
          4C       1.00      0.72      0.84       831

    accuracy                           0.77      1607
   macro avg       0.78      0.80      0.76      1607
weighted avg       0.85      0.77      0.78      1607

Epoch 1 Step 41 ave_loss 0.1943
Epoch 1 Step 83 ave_loss 0.1184
Epoch 1 Step 125 ave_loss 0.0773
Epoch 1 Step 167 ave_loss 0.0492
Epoch 1 Step 209 ave_loss 0.0619
              precision    recall  f1-score   support

          2C       0.87      0.91      0.89       409
          3C       0.88      0.96      0.92       367
          4C       1.00      0.93      0.97       831

    accuracy                           0.93      1607
   macro avg       0.

In [ ]:
#VGG19
main('vgg19')

In [ ]:
#RESNEET50
main('resnet50')